In [9]:

import torch
print(torch.version.cuda)  # sollte '12.2' oder ähnlich anzeigen
print(torch.cuda.is_available())  # sollte True sein
print(torch.cuda.current_device())  # sollte 0 sein, wenn eine GPU verfügbar ist


import d3rlpy
print(d3rlpy.__version__)

!nvidia-smi

!nvcc --version

12.1
True
0
2.8.1
Mon Nov 10 11:11:57 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 538.78                 Driver Version: 538.78       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  GRID P4-8A                   WDDM  | 00000000:06:10.0 N/A |                  N/A |
| N/A    0C    P8              N/A /  N/A |   1011MiB /  8192MiB |     N/A      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+-------

In [ ]:
import d3rlpy
import numpy as np


# vector observation
# 1000 steps of observations with shape of (100,)
observations = np.random.random((1000, 100))

# 1000 steps of actions with shape of (4,)
actions = np.random.random((1000, 4))

# 1000 steps of rewards
rewards = np.random.random(1000)

# 1000 steps of terminal flags
terminals = np.random.randint(2, size=1000)

dataset = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,
)


# if you don't use GPU, set device=None instead.
sac = d3rlpy.algos.SACConfig().create(device='cuda:0')

# initialize neural networks with the given observation shape and action size.
# this is not necessary when you directly call fit or fit_online method.
sac.build_with_dataset(dataset)
# calculate metrics with training dataset
td_error_evaluator = d3rlpy.metrics.TDErrorEvaluator(episodes=dataset.episodes)

# evaluate algorithm on the environment
#rewards = env_evaluator(sac, dataset=None)

sac.fit(
    dataset,
    n_steps=10000,
    evaluators={
        'td_error': td_error_evaluator,
    }
)

# get first observation from the dataset
observation = observations[0]

# return actions based on the greedy-policy
action = sac.predict(np.expand_dims(observation, axis=0))

# estimate action-values
value = sac.predict_value(np.expand_dims(observation, axis=0), action)

# save full parameters and configurations in a single file.
sac.save('sac.d3')
# load full parameters and build algorithm
sac2 = d3rlpy.load_learnable("sac.d3")

# save full parameters only
sac.save_model('sac.pt')

# save the greedy-policy as TorchScript
sac.save_policy('policy.pt')

SyntaxError: invalid syntax (707325908.py, line 4)